In [1]:
import pandas as pd
import os

In [2]:
fileS = os.path.join('schools_complete.csv')
schools_df = pd.read_csv(fileS)
fileT = os.path.join('students_complete.csv')
students_df = pd.read_csv(fileT)

In [3]:
count_schools = len(schools_df)
tot_budget = sum(schools_df.budget)
count_students = len(students_df)
average_math = students_df["math_score"].mean()
average_read = students_df["reading_score"].mean()
passing_math = (len(students_df.loc[students_df["math_score"] > 70, : ])/count_students)*100
passing_read = (len(students_df.loc[students_df["reading_score"] > 70, : ])/count_students)*100
passing_overall = (passing_math + passing_read)/2

In [4]:
# District Summary
schools_total_df = pd.DataFrame({"Total Schools": [count_schools], 
                                 "Total Budget": [tot_budget],
                                 "Total Students": [count_students],
                                 "Average Math Score": [average_math],
                                 "Average Reading Score": [average_read],
                                 "% Passing Math": [passing_math],
                                 "% Passing Reading": [passing_read],
                                 "Overall Passing Rate": [passing_overall]})
schools_total_df["Total Budget"] = schools_total_df["Total Budget"].map("${0:,.0f}".format)
schools_total_df

,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Passing Rate,Total Budget,Total Schools,Total Students
0,72.392137,82.971662,78.985371,81.87784,77.681899,"$24,649,428",15,39170


In [80]:
schools_df.rename(columns={"name" : "school"}, inplace=True)
#schools_df.head()
# merge students and schools on school name
stud_school_table = pd.merge(schools_df, students_df, on="school")
#stud_school_table.head()
#Calculate total of students per school
stud_school_df = pd.DataFrame(stud_school_table["school"].value_counts())
stud_school_df.reset_index(inplace=True)
stud_school_df.columns = ["School","Total Students"]
stud_school_df

#Calculate Budget per Schoo
school_budget_dict = { "School": schools_df["school"],
                       "Total School Budget": schools_df["budget"]}
school_budget = pd.DataFrame(school_budget_dict)
#school_budget
stud_school_stat = pd.DataFrame(stud_school_df)
stud_school_stat = pd.merge(stud_school_df,school_budget, on="School")
#stud_school_stat

#Calculate School Type
school_type_dict = { "School": schools_df["school"],
                     "School Type": schools_df["type"]}
school_type = pd.DataFrame(school_type_dict)
#school_type
stud_school_stat = pd.merge(stud_school_stat,school_type, on="School")
#stud_school_stat

#Calculate Budget per School per Student 
stud_school_stat["Per Stdnt Budget"] = stud_school_stat["Total School Budget"]/stud_school_stat["Total Students"]
per_stud_budget = stud_school_stat["Per Stdnt Budget"]
#Calculate Average Math Score per School
stud_school_math = pd.DataFrame(stud_school_table.groupby("school")["math_score"].mean())
stud_school_math.reset_index(inplace=True)
stud_school_math.columns=["School", "Average Math Score"]
#stud_school_math
stud_school_stat = pd.merge(stud_school_stat, stud_school_math, on="School")

#Change format to $ dollar sign and decimals to Per Student Budget and Total School Budget columns
stud_school_stat["Per Stdnt Budget"] = stud_school_stat["Per Stdnt Budget"].map("${0:,.2f}".format)
stud_school_stat["Total School Budget"] = stud_school_stat["Total School Budget"].map("${0:,.2f}".format)
#stud_school_stat

#Calculate Average Reading Score
stud_school_read = pd.DataFrame(stud_school_table.groupby("school")["reading_score"].mean())
stud_school_read.reset_index(inplace=True)
stud_school_read.columns=["School", "Average Reading Score"]
#stud_school_read
stud_school_stat = pd.merge(stud_school_stat, stud_school_read, on="School")
#stud_school_stat

#Calculate Percent of Students passing math with a score greater than 70
passing_math = (students_df.loc[students_df["math_score"] > 70, : ]).groupby("school")["math_score"].count()
stud_by_school = students_df.loc[students_df["Student ID"]].groupby("school")["Student ID"].count()
percent_passing_math = pd.DataFrame((passing_math/stud_by_school)*100)
percent_passing_math.reset_index(inplace=True)
percent_passing_math.columns=["School", "Percent Passing Math"]
stud_school_stat = pd.merge(stud_school_stat, percent_passing_math, on="School")
#stud_school_stat
#Calculate Percent of Students passing math with a score greater than 70
passing_reading = (students_df.loc[students_df["reading_score"] > 70, : ]).groupby("school")["reading_score"].count()
percent_passing_reading = pd.DataFrame((passing_reading/stud_by_school)*100)
percent_passing_reading.reset_index(inplace=True)
percent_passing_reading.columns=["School", "Percent Passing Reading"]
stud_school_stat = pd.merge(stud_school_stat, percent_passing_reading, on="School")
#stud_school_stat
overall_passing_rate = (stud_school_stat["Percent Passing Math"] + stud_school_stat["Percent Passing Reading"])/2
stud_school_stat["Overall Passing Rate"] = overall_passing_rate
stud_school_stat

,School,Total Students,Total School Budget,School Type,Per Stdnt Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
0,Bailey High School,4976,"$3,124,928.00",District,$628.00,77.048432,81.033963,64.630225,79.300643,71.965434
1,Johnson High School,4761,"$3,094,650.00",District,$650.00,77.072464,80.966394,63.852132,78.281874,71.067003
2,Hernandez High School,4635,"$3,022,020.00",District,$652.00,77.289752,80.934412,64.746494,78.187702,71.467098
3,Rodriguez High School,3999,"$2,547,363.00",District,$637.00,76.842711,80.744686,64.066017,77.744436,70.905226
4,Figueroa High School,2949,"$1,884,411.00",District,$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
5,Huang High School,2917,"$1,910,635.00",District,$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
6,Ford High School,2739,"$1,763,916.00",District,$644.00,77.102592,80.746258,65.753925,77.510040,71.631982
7,Wilson High School,2283,"$1,319,574.00",Charter,$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
8,Cabrera High School,1858,"$1,081,356.00",Charter,$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
9,Wright High School,1800,"$1,049,400.00",Charter,$583.00,83.682222,83.955000,90.277778,93.444444,91.861111


In [8]:
#Create a table that highlights the top 5 performing schools based on Overall Passing Rate. Include:
top_performing_schools = pd.DataFrame(stud_school_stat)
top_performing_schools.reset_index(inplace=True)
top_performing_schools.sort_values("Overall Passing Rate", ascending=False, inplace=True)
top_performing_schools.head(5)

,level_0,index,School,Total Students,Total School Budget,School Type,Per Stdnt Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
0,0,7,Wilson High School,2283,"$1,319,574.00",Charter,$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
1,1,13,Pena High School,962,"$585,858.00",Charter,$609.00,83.839917,84.044699,91.683992,92.203742,91.943867
2,2,9,Wright High School,1800,"$1,049,400.00",Charter,$583.00,83.682222,83.955000,90.277778,93.444444,91.861111
3,3,8,Cabrera High School,1858,"$1,081,356.00",Charter,$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
4,4,14,Holden High School,427,"$248,087.00",Charter,$581.00,83.803279,83.814988,90.632319,92.740047,91.686183


In [12]:
#Create a table that highlights the bottom 5 performing schools based on Overall Passing Rate. Include all of the same metrics as above.
bottom_performing_schools = pd.DataFrame(stud_school_stat)
#bottom_performing_schools.reset_index(inplace=True)
bottom_performing_schools.sort_values("Overall Passing Rate", ascending=True, inplace=True)
bottom_performing_schools.head(5)


,level_0,index,School,Total Students,Total School Budget,School Type,Per Stdnt Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
14,14,3,Rodriguez High School,3999,"$2,547,363.00",District,$637.00,76.842711,80.744686,64.066017,77.744436,70.905226
13,13,5,Huang High School,2917,"$1,910,635.00",District,$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
12,12,1,Johnson High School,4761,"$3,094,650.00",District,$650.00,77.072464,80.966394,63.852132,78.281874,71.067003
11,11,4,Figueroa High School,2949,"$1,884,411.00",District,$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
10,10,2,Hernandez High School,4635,"$3,022,020.00",District,$652.00,77.289752,80.934412,64.746494,78.187702,71.467098


In [41]:
#Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
aver_math_grade = pd.DataFrame(students_df)
aver_math_grade = (aver_math_grade.groupby(['school','grade'])['math_score'].mean()).unstack(level=1)
aver_math_grade

grade,10th,11th,12th,9th
school,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857


In [42]:
#Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
aver_reading_grade = pd.DataFrame(students_df)
aver_reading_grade = (aver_reading_grade.groupby(['school','grade'])['reading_score'].mean()).unstack(level=1)
aver_reading_grade

grade,10th,11th,12th,9th
school,,,,
Bailey High School,80.907183,80.945643,80.912451,81.303155
Cabrera High School,84.253219,83.788382,84.287958,83.676136
Figueroa High School,81.408912,80.640339,81.384863,81.198598
Ford High School,81.262712,80.403642,80.662338,80.632653
Griffin High School,83.706897,84.288089,84.013699,83.369193
Hernandez High School,80.660147,81.396140,80.857143,80.866860
Holden High School,83.324561,83.815534,84.698795,83.677165
Huang High School,81.512386,81.417476,80.305983,81.290284
Johnson High School,80.773431,80.616027,81.227564,81.260714


In [87]:
#Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. 
#Include in the table each of the following:
raw_data = {"Spending": per_stud_budget,
            "Avg Math": stud_school_stat["Average Math Score"],
            "Avg Read": stud_school_stat["Average Reading Score"],
            "Pctg Pass Math": stud_school_stat["Percent Passing Math"],
            "Pctg Pass Read": stud_school_stat["Percent Passing Reading"],
            "Overall Pass Rate": stud_school_stat["Overall Passing Rate"]}
perf_spend_df = pd.DataFrame(raw_data)
bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
perf_spend_df.reset_index()
pd.cut(perf_spend_df["Spending"], bins, labels=group_names)
perf_spend_df["Spending Summary"] = pd.cut(perf_spend_df["Spending"],
                                           bins, labels=group_names)
#perf_spend_df

In [86]:
# Creating a group based off of the bins
spending_groups = perf_spend_df.groupby("Spending Summary")
spending_groups.max()

,Avg Math,Avg Read,Overall Pass Rate,Pctg Pass Math,Pctg Pass Read,Spending
Spending Summary,,,,,,
<$585,83.803279,83.989488,92.093736,90.932983,93.864370,583.0
$585-615,83.839917,84.044699,91.943867,91.683992,92.617831,609.0
$615-645,83.418349,83.848930,91.559633,90.214067,93.392371,644.0
$645-675,77.289752,81.182722,71.467098,64.746494,78.813850,655.0


In [106]:
#Scores by School Size
#Repeat the above breakdown, but this time group schools based on a reasonable approximation of school size 
#(Small, Medium, Large).
stud_school_stat["SchSize"] = schools_df["size"]
raw_data = {"School Size": stud_school_stat["SchSize"],
            "Avg Math": stud_school_stat["Average Math Score"],
            "Avg Read": stud_school_stat["Average Reading Score"],
            "Pctg Pass Math": stud_school_stat["Percent Passing Math"],
            "Pctg Pass Read": stud_school_stat["Percent Passing Reading"],
            "Overall Pass Rate": stud_school_stat["Overall Passing Rate"]}
school_size_df = pd.DataFrame(raw_data)
bins = [0, 1000, 2000, 5000]
group_names = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]
school_size_df.reset_index()
pd.cut(school_size_df["School Size"], bins, labels=group_names)
school_size_df["School Size Summary"] = pd.cut(school_size_df["School Size"],bins, labels=group_names)
#school_size_df
# Creating a group based off of the bins
school_size_groups = school_size_df.groupby(["School Size Summary"])
school_size_groups.max()

,Avg Math,Avg Read,Overall Pass Rate,Pctg Pass Math,Pctg Pass Read,School Size
School Size Summary,,,,,,
Small(<1000),83.682222,83.975780,91.861111,90.277778,93.864370,962
Medium(1000-2000),83.803279,83.814988,91.686183,90.632319,92.740047,1858
Large(2000-5000),83.839917,84.044699,92.093736,91.683992,93.392371,4976


In [110]:
stud_school_stat["SchType"] = schools_df["type"]
raw_data = {"School Type": stud_school_stat["SchType"],
            "Avg Math": stud_school_stat["Average Math Score"],
            "Avg Read": stud_school_stat["Average Reading Score"],
            "Pctg Pass Math": stud_school_stat["Percent Passing Math"],
            "Pctg Pass Read": stud_school_stat["Percent Passing Reading"],
            "Overall Pass Rate": stud_school_stat["Overall Passing Rate"]}
school_type_df = pd.DataFrame(raw_data)
#bins = ["Charter","District"]
#group_names = ["Charter", "District"]
#school_type_df.reset_index()
#pd.cut(school_type_df["School Type"], labels=group_names)
#school_type_df["School Type Summary"] = pd.cut(school_size_df["School Type"], labels=group_names)
#school_type_df = (school_type_df.groupby(['School Type']))
#school_type_df.head()
school_type_groups = school_type_df.groupby(["School Type"])
school_type_groups.mean()

,Avg Math,Avg Read,Overall Pass Rate,Pctg Pass Math,Pctg Pass Read
School Type,,,,,
Charter,80.205047,82.436613,81.471365,77.241274,85.701456
District,80.692510,82.634988,83.012576,79.299045,86.726108
